# RFA Trabjo Prácticas: DeepGlobe-Road-Extraction con DataAgumentation

# Descripción del proyecto
Este proyecto pretente abordar el problema de la `road extraction` / `road segmentataion`. Es decir, a partir de imágenes satelitáles, extrar en que lugares hay carrertas y qué lugares no. O, mejor dicho: predecir para cualquier imagen satelital, qué lugares tienen carretera (`1`) y qué lugares son solo 'fondo' (`0`). La tarea se podría comparar (a menor escala) a lo que tuvo que haber hecho Google al hacer los mapas de carreras en Google Maps.

El proyecto se ha enfocado con la idea de, a partir de un dataset, y tras generar las particiones de datos correspondientes: entrenar diferentes modelos con distintas configuraciones, con la idea de encontrar cuál es la que mejores resultados obtiene en la tarea. Tras entrenar todos los modelos, se pondrán a prueba con un conjunto de datos reservado para test, del que se podrán sacar conclusiones.

En concreto, se probarán diferentes arquitecturas, diferentes funciones de pérdida y sobre todo, diferentes aumentos de datos.

# Dataset
El dataset base utilizado es el `DeepGlobe Road Extraction`. Se ha particionado de este dataset, se ha escogido la partición de entranamiento únicamente por ser la única totalmente etiquetada. A partir de ella se han creado otros subconjunto de entrenamiento / validación / test en un 70%, 15%, 15% respectivamente. 

Se ha elegido este dataset por estar bastate bien documentado, con suficientes referencias y con un tamaño de datos elevado.

# Data agumentation
Esta técnica es uno de los puntos más interesantes del proyecto, ya que permite ampliar la variedad de ejemplos que ve el modelo y simular condiciones reales (*rotaciones, espejado, cambios de color, ruido, recortes, etc.)*, lo que suele mejorar la capacidad de generalización y la robustez de los modelos.

### Ventajas
- Aumenta la diversidad de datos sin etiquetar más imágenes reales, reduciendo overfitting y reduciendo el coste que implica conseguir más datos.
- Simula situaciones y variaciones (*ángulo, iluminación, ruido*), útil para modelos que deben generalizar a nuevas zonas no vistas.
- Algunas transformaciones (*shift de color, ruido*) ayudan a que el modelo no dependa de colores/contrastes específicos de una región.

### Desventajas / riesgos
- Si la augmentación es demasiado agresiva, puede romper la coherencia entre imagen y máscara (*p. ej. recortes que cortan carreteras importantes*) y degradar la calidad del aprendizaje.
- Aumentos no realistas (*colores extremos, formas artificiales*) pueden enseñar al modelo patrones que no existen en el mundo real y empeorar la generalización.
- Augmentaciones que cambian geometría (*ej. warps fuertes*) pueden alterar la topología de la red de carreteras y producir predicciones “fantasma”.
- Es díficil saber si aplicamos mucha o poca augmentación.

### Mucho versus poco aumento
- Poca augmentación: riesgo de sobreajuste a las condiciones del dataset (*p. ej. mismas zonas, mismas horas/delimitaciones*). Esto termina en un mejor rendimiento en datos muy parecidos al train, pero peor en nuevos dominios.
- Mucha augmentación: mejora robustez en escenarios variados, pero puede empeorar convergencia y precisión si las transformaciones desnaturalizan las señales relevantes (*anchura/contraste de carreteras*). Suele haber una ventana óptima de intensidad/probabilidad de aplicar cada transformación.

### Adversarial learning y augmentación `?????????????????????'`
Una parte importante del estudio en el aprendizaje automatico es el uso de `estrategias adversariales`. Estas conllevan entrenar un discriminador que diferencie segmentaciones reales de generadas, o utilizar perturbaciones adversariales durante entrenamiento. Esta téncia ha mostrado mejorar robustez y, en algunos trabajos, la conectividad y calidad de las máscaras en tareas de segmentación semántica y extracción de vías. Ejemplos y revisiones muestran enfoques semi-supervisados y basados en GANs aplicados a segmentación y road extraction. 
faculty.ucmerced.edu

### Transformaciones aplicada (y por qué tienen sentido)
- **ROTATE**: rotaciones aleatorias: ayuda a invariancia rotacional. Útil si las carreteras aparecen en cualquier orientación.
- **MIRROR**: espejado horizontal/vertical. Aumenta variación geométrica sin cambiar etiquetas.
- **SUB**: sub-crop (recorte). Muestra patches locales y obliga al modelo a decidir con contexto reducido. *Hay que tener cuidado ya que las imágenes pueden perder la continuidad*.
- **CIRCLES**: añadir círculos con color igual a la media de la imagen. Se usa la estrategia de elegir parches con un color 'homogeneo' para evitar introducir artefactos negros que el modelo asocie erróneamente con carreteras (p*or ejemplo sombras grandes*). Usar el color medio reduce el riesgo de introducir un sesgo cromático artificial y ayuda a que el modelo no aprenda “negro  = carretera”. *Decimos negro pero con cualquier color fijo se aplica la misma lógica*.
- **SHIFT_COLOR**: variación del tono. Simula diferentes condiciones de iluminación/sensores.
- **NOISE**: añadir ruido. Mejora robustez frente a compresión de imágenes y sensores ruidosos.

Se aplica cada transformación con una probabilidad razonable `(p.ej. 0.2–0.5)` y, en general, controla la fuerza de la transformación por validación. Registrar qué combinaciones/probabilidades funcionan mejor es tan importante como el propio experimento.

# Modelos
Se pretende utilizar cuatro arquitecturas diferentes, las cuales podemos separar en dos tipos: Redes neuronales Convolucionales y Vision transformers. Del primer tipo solo tenemos un modelo, pero será suficiente para compara la versatilidad de estos en tareas de visión.

En concreto, se usarán estas arquitéctuas:

- **U-Net**: Arquitectura en forma de U con camino de contracción (encoder) y expansión (decoder) y skip connections que recuperan detalles espaciales finos. Fue diseñada para segmentación con pocos datos y enfatiza el uso de augmentación y entrenamiento end-to-end para obtener buena localización. (Ronneberger et al., 2015). 
arXiv

- **FPN (Feature Pyramid Network)**: Construye una pirámide de características multi-escala mediante un camino top-down con conexiones laterales; permite explotar jerarquías de la CNN para detectar/segmentar objetos de distintos tamaños con coste marginal. Muy útil para captar carreteras de diferentes anchuras. (Lin et al., CVPR 2017). 
CVF Open Access

- **PSPNet (Pyramid Scene Parsing Network)**: Introduce un módulo de pyramid pooling que agrega información de contexto global a distintas escalas (regiones grandes→pequeñas), mejorando la coherencia semántica y la predicción en escenas complejas. Bueno para incorporar contexto amplio (p. ej. distinguir carretera de parche sombreado). (Zhao et al., CVPR 2017). 
CVF Open Access

- **ViT / Transformers para segmentación**:  Aplican self-attention a parches de imagen para modelar dependencias globales desde las primeras capas; pueden usarse como encoder puro (SETR) o híbrido (encoder transformer + decoder convolucional). Mejora el contexto global y las relaciones a larga distancia, útil cuando la estructura de la red de carreteras depende de contexto amplio. (SETR — Zheng et al., 2021). 
arXiv


### Funciones de pérdida 
En un experimetno así, probar varias funciones de pérdida es útil porque la pérdida `guía el aprendizaje`: cambia qué errores el modelo prioriza, cómo se comportan los gradientes y, en la práctica, afecta a la precisión, la robustez y la conectividad de las carreteras predichas. Las métricas que usadas en el proyecto han sido:

- **DiceLoss**: Basada en la métrica Dice / F1; común y robusta frente al desequilibrio de clases porque maximiza la superposición entre predicción y etiqueta. Muy usada en segmentación médica y en muchos trabajos de segmentación de carreteras (uso clásico: V-Net / Dice). 
arXiv

- **BCEWithLogitsLoss**: Binary Cross Entropy aplicada sobre logits (incluye la sigmoid internamente, más numéricamente estable). Es adecuada para problemas binarios pero puede sufrir cuando las clases están muy desbalanceadas, ya que penaliza por píxel y la mayoría son fondos. (PyTorch docs explican el uso y estabilidad). 
docs.pytorch.org

- **BCE + Dice (composite loss)**: Combinación práctica y extendida: la BCE aporta gradientes más “suaves” y la Dice compensa el desequilibrio maximizando la métrica objetivo. Por eso se suele entrenar con una pérdida compuesta Loss = BCEWithLogits + α·DiceLoss (α en [0.5,1] típico). La justificación práctica: la BCE sola no porque tiene problemas con balanceo de clases y pues la mayoría NO es road, es background entonces...

- **IoU (Intersectoin over Union)**: La IoU calcula la superficie que ha sido bien predicha, entre la cantidad total de superficia que ha sido predicha entre la referencia y el modelo. Esto crea una proporción de 'entre todo lo que había, cuanto ha sido bien predicho'. Está función de pérdida no se usa para entrenamiento sino pás bien para hacer el testing. Algunas referencias la marcan como una función que puede manejar mejor el desequilibrio, pero la combinación BCE+Dice es un buen punto de partida y frecuente en la literatura para el entrenamiento. Es por ello que NO la usaremos para entrenar pero sí para hacer testing. Una vez todos los modelos estén entrenado, será nuestra métrica. 



# Trabajos de referencia `¿¿¿¿¿¿¿¿¿¿¿¿¿¿¡¡¡`
Hung, W.-C. et al., Adversarial Learning for Semi-Supervised Semantic Segmentation (BMVC 2018) — adversarial training aplicado a segmentación, útil para semi-supervisión y robustez. 
faculty.ucmerced.edu

Chen, H. et al., SW-GAN / Semi-weakly supervised methods for road extraction (MDPI) — ejemplos de GANs y aprendizaje adversarial aplicados a extracción de carreteras. 
MDPI

Milletari, V. et al., V-Net (2016) — introducción / uso del Dice loss en segmentación (referencia clásica). 
arXiv

PyTorch — documentación de BCEWithLogitsLoss para referencia técnica sobre la implementación estable numéricamente. 
docs.pytorch.org

Liu, R. et al., A Review of Deep Learning-Based Methods for Road Extraction (Remote Sensing, 2024) — revisión reciente que te sirve para contextualizar técnicas actuales y comparativas. 
MDPI

# Implementación del código

Todo el código se encuentra en este repositorio, más en concreto de forma accesible en [app](../app/main.py). En el [README.md](../README.md) se explica con más detalles como usar el repositorio y todo su fucionamiento.

En resumidas cuentas, se usa [main.py](../app/main.py) como punto de entrada a la aplicación para llamar a todos los scripts, y desde ahí se orquesta toda la funcionalidad del repositorio.

# Análisis de los resultados

### Configuración del notebook

In [ ]:
import os

print(os.getcwd())
if not os.getcwd().endswith("app"):
    os.chdir("../app")
    print(os.getcwd())

%load_ext autoreload
%autoreload 2

### Carga de modelos

In [ ]:
from src.config import Configuration
from maikol_utils.file_utils import list_dir_files
from src.utils import PathParser

CONFIG = Configuration()

files, n = list_dir_files(CONFIG.LOGS_FOLDER, recursive=True)
models = [f for f in files if "checkpoints" in f]
metrics = [f for f in files if "test_metrics" in f]
    
model_obj = [PathParser(m) for m in models]
metrics_obj = [PathParser(m) for m in metrics]
model_metrics = [(mod, met) for mod in model_obj for met in metrics_obj if mod.name == met.name]

len(model_obj)
len(metrics_obj)
len(model_metrics)

### Tiempo de entrenamiento

In [ ]:
from src.script_refactor import get_minutes

model_hours = [(mod, get_minutes(met.path)) for mod, met in model_metrics]

In [ ]:
import matplotlib.pyplot as plt

# Separate groups
vit_times = [m for mod, m in model_hours if "ViT" in mod.name]
cnn_times = [m for mod, m in model_hours if "ViT" not in mod.name]
all_times = [m for _, m in model_hours]

# Boxplot
plt.boxplot([all_times, vit_times, cnn_times], labels=["All", "ViT", "CNN"])
plt.ylabel("Training time (minutes)")
plt.title("Training time distribution per model type")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


<span style="color:LightGreen"> 
Vemos que no hay diferencia significativa entre los tiempos de entrenamietno
Hay algún que otro entrenamiento que toma mucho más tiempo, pero no de forma general
</span>

### Resultados de infernecia

In [ ]:
from maikol_utils.file_utils import load_json
from src.utils import plot_model_scores

cnn_models_scores = load_json(CONFIG.cnn_model_scores)
vit_models_scores = load_json(CONFIG.vit_model_scores)

all_models_scores = {**cnn_models_scores, **vit_models_scores}

plot_model_scores(all_models_scores)

In [ ]:
from src.utils import plot_model_scores_by_architecture

# Plot models grouped by architecture with data augmentation coloring
plot_model_scores_by_architecture(all_models_scores)

### Análisis con Box Plots

Los box plots nos permiten ver la distribución de IoU para diferentes grupos, mostrando mediana, cuartiles, y valores atípicos.

In [ ]:
# Importar la nueva función de box plots
from src.utils.visualizations import plot_iou_boxplots_by_parameter

In [ ]:
# Box plot agrupado por ARQUITECTURA
print("📊 Box Plot: IoU por Arquitectura")
print("=" * 50)
plot_iou_boxplots_by_parameter(all_models_scores, group_by='ARC')

In [ ]:
# Box plot agrupado por FUNCIÓN DE PÉRDIDA
print("📊 Box Plot: IoU por Función de Pérdida")
print("=" * 50)
plot_iou_boxplots_by_parameter(all_models_scores, group_by='LSS')

In [ ]:
# Box plot agrupado por DATA AUGMENTATION
print("📊 Box Plot: IoU por Data Augmentation")
print("=" * 50)
plot_iou_boxplots_by_parameter(all_models_scores, group_by='AUG')

# Análisis del data augmentation